In [1]:
import cv2
import mediapipe as mp

2023-12-08 20:36:23.207338: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-08 20:36:23.228404: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-08 20:36:23.228423: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-08 20:36:23.228952: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-08 20:36:23.232316: I tensorflow/core/platform/cpu_feature_guar

In [2]:
class SinglePoseEstimation():
    def __init__(self):
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_pose = mp.solutions.pose

        self.pose = self.mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
        
        self.frame = None
        self.result = None
        self.landmarks = None
        
    
    def set_frame(self, frame):
        self.frame = frame

    def verify_variable(self):
        # Check if the self.frame is not set yet
        if self.frame is None:
            print("Frame not being set yet")
            return

    def estimate(self):
        self.verify_variable()
        
        # Convert BGR to RGB
        image = cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Process image
        self.result = self.pose.process(image)
    
        # Convert RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image.flags.writeable = True
        
        # Extract feature
        try:
            self.landmarks = self.result.pose_landmarks.landmark
        except:
            pass
        
    def get_annotated_frame(self):
        self.verify_variable()
            
        # Render detection
        self.mp_drawing.draw_landmarks(self.frame, self.result.pose_landmarks, self.mp_pose.POSE_CONNECTIONS,
                                self.mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                self.mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))
        return self.frame
        

In [3]:
single_pose = SinglePoseEstimation()
cap = cv2.VideoCapture(0)


while True:
    ret, frame = cap.read()
    if not ret:
        break

    single_pose.set_frame(frame)
    single_pose.estimate()
   
    cv2.imshow("webcam", single_pose.get_annotated_frame())

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture object and close the window
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1702042584.109871   27436 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1702042584.144712   27524 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 510.39.01), renderer: NVIDIA GeForce RTX 3060/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
